In [1]:
import numpy as np
import h5py
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from RNNGRU.RNNnet import evaluate, EEGRNN

In [2]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train = np.load("X_train_valid.npy")
y_train = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

X_train = np.expand_dims(X_train, 3)
X_train = X_train.transpose(0,3,1,2)
X_train = X_train.astype('float32')
y_train=y_train-769
y_train = np.int_(y_train)
y_train=np.eye(4)[y_train]

X_test = np.expand_dims(X_test, 3)
X_test = X_test.transpose(0,3,1,2)
X_test = X_test.astype('float32')
y_test= y_test-769
y_test = np.int_(y_test)
y_test=np.eye(4)[y_test]

In [ ]:

rnn = EEGRNN()
criterion = nn.MSELoss()
optimizer = optim.Adagrad(rnn.parameters())
batch_size = 25
for epoch in range(10):  # loop over the dataset mdultiple times
    print("\nEpoch ", epoch)
    
    running_loss = 0.0
    
    for i in range((X_train.shape[0])//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.FloatTensor(y_train[s:e])
        
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = rnn(inputs)
        pred = outputs
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        
        running_loss += loss.item()
    
        # accuracy
    print(running_loss)
    print("Test Accuracy - ", evaluate(rnn, X_test, y_test))
    print("Train Accuracy- ", evaluate(rnn, X_train, y_train))


Epoch  0


/Users/zhijieyao/Desktop/Ucla/2019 winter/ECE 239/project/project/RNNGRU/RNNnet.py:184: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.sm(x)


13.774803094565868
Test Accuracy -  0.5620767494356659
